# ДЗ №3 
## Обучение моделей глубокого обучения на PyTorch

In [ ]:
!pip3 install torch torchvision numpy matplotlib

Ваша задача на этой неделе - повторить модель трёхслойного перцептрона из прошолго задания на PyTorch, разобрать лучшие практики обучения моделей глубокого обучения и провести серию экспериментов

In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from typing import Tuple, List, Type, Dict, Any

Для того, чтобы эксперимент можно было повторить, хорошей практикой будет зафиксировать генератор случайных чисел. Также, рекоммендуется зафиксировать RNG в numpy и, если в качестве бэкенда используется cudnn - включить детерминированный режим.

Подробнее: https://pytorch.org/docs/stable/notes/randomness.html

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Модель

Основным способом организации кода на PyTorch является модуль. Простые модели могут быть реализованны из готовых модулей ( к примеру, torch.nn.Sequential, torch.nn.Linear и т.д. ), для более сложных архитектур часто приходтся реализовывать собственные блоки. Это достаточно легко сделать - достаточно написать класс, наследуемый от torch.nn.Module и реализующий метод .forward, который принимает и возвращает тензоры ( torch.Tensor )

Пример реализации кастомного модуля из официальной документации: https://pytorch.org/tutorials/beginner/pytorch_with_examples.html3

#### Задание 1

Повторите реализацию трёхслойного перцептрона из предыдущего задания на pytorch. Желательно также, чтобы реализация модели имела параметризуемую глубину ( количество слоёв ), количество параметров на каждом слое и функцию активации

In [ ]:
class Perceptron(torch.nn.Module):
    
    def __init__(self, 
                 input_resolution: Tuple[int, int] = (28, 28),
                 input_channels: int = 1, 
                 hidden_layer_features: List[int] = [256, 256, 256],
                 activation: Type[torch.nn.Module] = torch.nn.ReLU,
                 num_classes: int = 10):

        super().__init__()
        raise NotImplementedError
    
    def forward(self, x):
        raise NotImplementedError

Следующий код позволяет посмотреть архитектуру получившейся модели и общее количество обучаемых параметров. Мы хотим, чтобы количество параметров в модели было порядка сотен тысяч - если у вас получается больше или меньше - попробуйте изменить архитектуру модели

In [ ]:
model = Perceptron()
print(model)
print('Total number of trainable parameters', 
      sum(p.numel() for p in model.parameters() if p.requires_grad))

### Обучающая выборка

На практике, наиболее важным для успеха обучения любой ML модели является этап подготовки данных. Модели глубокого обучения не являются исключением - большая, чистая и репрезантативная поставленной задаче обучающая выборка часто важнее, чем архитектура самой модели. В нашем случае мы используем качественный и проверенный временем MNIST, но в практических задачах часто будет получаться так, что лучшим способом добиться улучшения качества модели будет сбор дополнительных данных и очистка датасета.

### Предобработка данных

Для улучшения сходимости алгоритма обучения и качества полученной модели данные должны быть предобработанны:

1. Среднее каждой входной переменной близко к нулю
2. Переменные отмасштабированы таким образом, что их дисперсии примерно одинаковы ( из соображений вычислительной устойчивости, мы хотим, чтобы все величины по порядку величины были близки к еденице )
3. По возможности, входные переменные не должны быть скоррелированны. Важнось этого пункта в последние годы ставится под сомнение, но всё-же в некоторых случаях это может влиять на результат

Подробнее можно почитать здесь: http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf

### Аугументация обучающей выборки

В зависимости от задачи, мы можем применять к обучающей выборке различные преобразования, которые позволят увеличить эффективный размер выборки без дополнительной разметки. К примеру, для задачи классификации кошек и собак мы можем зеркально отразить изображение вокруг вертикальной оси - при этом класс изображения не изменится, и само изображение останется по прежнему будет принадлежать исходному распределению. Есть много разных техник аугументации данных, и их применимость и эффективность сильно зависит от датасета и задачи. 

Подробнее можно почитать здесь: https://link.springer.com/content/pdf/10.1186/s40537-019-0197-0.pdf

### Задание 2

Обоснуйте, почему аугументация обучающей выборки позволяет добиться прироста качества модели, несмотря на то, что она не добавляет в неё дополнительную информацию.

...

### Задание 3

Какие осмысленные аугментации вы можете придумать для следующих наборов данных:

1. Датасет изображений животных, размеченый на вид животного
2. Датасет с аудиозаписями голоса, размечеными на язык говорящего
3. Датасет cо значениями показаний датчиков температуры, влажности и давления с одной из метеостанций, размеченый на наличие или отсутствие осадков

...

### Задание 4

Напишите пайплайн для предобработки и аугументации данных. В torchvision.transforms есть готовые реализации большинства распространённых техник, если вы хотите добавить что-то своё, вы можете воспользоваться torchvision.transforms.Lambda. Также, для пайплайн для валидационной выборки не должен включать аугментации - мы хотим оценить качество модели на оригинальных данных.

Одним из обязательных шагов в вашем пайплайне должен быть torchvision.transforms.ToTensor(), который сконвертирует PIL изображение в torch.Tensor. 

In [ ]:
train_transforms = torchvision.transforms.Compose([
    # your core here
])

val_transforms = torchvision.transforms.Compose([
    # your core here
])

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='./mnist', 
                                           train=True, 
                                           download=True,
                                           transform=train_transforms)

val_dataset = torchvision.datasets.MNIST(root='./mnist', 
                                         train=False, 
                                         download=True, 
                                         transform=val_transforms)

Перед тем как запускать обучение всегда стоит посмотреть на данные после предобработки, и удостовериться, что они соответствуют ожидаемым

In [ ]:
indices = np.random.randint(0, len(train_dataset), size=256)

fig, axes = plt.subplots(nrows=16, ncols=16, figsize=(32, 32))
for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        sample_index = indices[i*16+j]
        sample, label = train_dataset[sample_index]
        ax.imshow(sample.cpu().numpy().transpose(1, 2, 0))
        ax.set_title(label)

### Обучение модели

Теперь, когда мы реализовали модель и подготовили данные мы можем приступить к непосредственному обучению модели. Костяк функции обучения написан ниже, далее вы должны будете реализовать ключевые части этого алгоритма

In [ ]:
def train_model(model: torch.nn.Module, 
                train_dataset: torch.utils.data.Dataset,
                val_dataset: torch.utils.data.Dataset,
                loss_function: torch.nn.Module = torch.nn.CrossEntropyLoss(),
                optimizer_class: Type[torch.optim.Optimizer] = torch.optim,
                optimizer_params: Dict = {},
                initial_lr = 0.01,
                lr_scheduler_class: Any = torch.optim.lr_scheduler.ReduceLROnPlateau,
                lr_scheduler_params: Dict = {},
                batch_size = 64,
                max_epochs = 1000,
                early_stopping_patience = 20
):
    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr, **optimizer_params)
    lr_scheduler = lr_scheduler_class(optimizer, **lr_scheduler_params)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

    best_val_loss = None
    best_epoch = None
    
    for epoch in range(max_epochs):
        
        print(f'Epoch {epoch}')
        train_single_epoch(model, optimizer, loss_function, train_loader)
        val_metrics = validate_single_epoch(model, loss_function, val_loader)
        print(f'Validation metrics: \n{val_metrics}')

        lr_scheduler.step(val_metrics['loss'])
        
        if best_val_loss is None or best_val_loss > val_metrics['loss']:
            print(f'Best model yet, saving')
            best_val_loss = val_metrics['loss']
            best_epoch = epoch
            torch.save(model, './best_model.pth')
            
        if epoch - best_epoch > early_stopping_patience:
            print('Early stopping triggered')
            return

### Задание 5

Реализуйте функцию производящую обучение сети на протяжении одной эпохи ( полного прохода по обучающей выборке ). На вход будет приходить модель, оптимизатор, функция потерь и DataLoader. При итерировании по data_loader вы будете получать пары вида ( данные, класс )

In [ ]:
def train_single_epoch(model: torch.nn.Module,
                       optimizer: torch.optim.Optimizer, 
                       loss_function: torch.nn.Module, 
                       data_loader: torch.utils.data.DataLoader):
    
    raise NotImplementedError

### Задание 6

Реализуйте функцию производящую рассчёт функции потерь на валидационной выборке.  На вход будет приходить модель, функция потерь и DataLoader. На выходе ожидается словарь с вида:
```
{
    'loss': <среднее значение функции потерь>,
    'accuracy': <среднее значение точности модели>
}
```

In [ ]:
def validate_single_epoch(model: torch.nn.Module,
                          loss_function: torch.nn.Module, 
                          data_loader: torch.utils.data.DataLoader):
    
    raise NotImplementedError

Если вы корректно реализовали все предыдущие шаги и ваша модель имеет достаточное количество обучаемых параметров, то в следующей ячейке должен пойти процесс обучения, и мы должны достичь итоговой accuracy выше 90%

In [ ]:
train_model(model, 
            train_dataset=train_dataset, 
            val_dataset=val_dataset, 
            loss_function=torch.nn.CrossEntropyLoss(), 
            initial_lr=0.01)

### Задание 7

Попытайтесь модифицировать процесс обучения таким образом, чтобы достигнуть наилучшего качества на валидационной выборке. Модель должна оставаться N-слойным перцептроном с количеством обучаемых параметров <= 500000. Для обучения разрешается использовать только датасет MNIST. Процесс обучения вы можете изменять по собственному усмотрению. К примеру, вы можете менять:

* Архитектуру модели, в рамках наложенных ограничений
* Функции активации в модели
* Используемый оптимизатор
* Расписание LR
* Сэмплинг данных при обучении ( e.g. hard negative mining)

В результате мы ожидаем увидеть код экспериментов и любые инсайты, которые вы сможете получить в процессе